In [1]:
#notebook for exploring and preprocessing the NYT bestseller list dataset
# obtained from https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers/version/1
# originally downloaded from NYT
# there is a kernel associated with this dataset, I use some methods from it
# the use of Levenshtein distance for checking integrity is my work

import json
import pandas as pd
import numpy as np

import re

from string import ascii_lowercase
from datetime import datetime

import Levenshtein as lev

In [ ]:
#import sys
#!{sys.executable} -m pip install python-Levenshtein

In [2]:
file = 'nyt2.json'
nyt_best = pd.read_json(file, lines=True, orient='columns')
nyt_best.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,{'$oid': '5b4aa4ead3089013507db18b'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",{'$numberInt': '27'},Bantam,ODD HOURS,{'$numberInt': '1'},{'$numberInt': '0'},{'$numberInt': '1'}
1,{'$oid': '5b4aa4ead3089013507db18c'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,{'$numberInt': '2'},{'$numberInt': '1'},{'$numberInt': '3'}
2,{'$oid': '5b4aa4ead3089013507db18d'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,{'$numberDouble': '24.95'},St. Martin's,LOVE THE ONE YOU'RE WITH,{'$numberInt': '3'},{'$numberInt': '2'},{'$numberInt': '2'}
3,{'$oid': '5b4aa4ead3089013507db18e'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,{'$numberDouble': '22.95'},Putnam,THE FRONT,{'$numberInt': '4'},{'$numberInt': '0'},{'$numberInt': '1'}
4,{'$oid': '5b4aa4ead3089013507db18f'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,{'$numberDouble': '24.95'},Doubleday,SNUFF,{'$numberInt': '5'},{'$numberInt': '0'},{'$numberInt': '1'}


In [3]:
#following is from kaggle kernel
# https://www.kaggle.com/tobaotic/clean-data



#convert string to lowercase and replace all non alphanumeric characters
def get_lc_values(v):
    regex = re.compile('[^a-z0-9]')
    lc_values = []
    for x in v:
        #this strips away all whitespace and makes everything lowercase
        lc_values.append(regex.sub('', x.lower()))

    return lc_values



def flatten_json(y):
    # extract values from json string in dataframe
    
    out = {}
    ret_val = '-'
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        elif type(x) is str:
            out[name[:-1]] = x


    flatten(y)

    if type(out) is dict:
        for v in out:
            ret_val = out[v]
    
    return ret_val


In [4]:
#also from kernel

for i in range(len(nyt_best)):
    nyt_best.loc[i]["_id"] = flatten_json(nyt_best.loc[i]["_id"])
    nyt_best.loc[i]["bestsellers_date"] = datetime.fromtimestamp(int(flatten_json(nyt_best.loc[i]["bestsellers_date"])[:10])).strftime('%Y-%m-%d')
    nyt_best.loc[i]["price"] = flatten_json(nyt_best.loc[i]["price"])
    nyt_best.loc[i]["published_date"] = datetime.fromtimestamp(int(flatten_json(nyt_best.loc[i]["published_date"])[:10])).strftime('%Y-%m-%d')
    nyt_best.loc[i]["rank"] = flatten_json(nyt_best.loc[i]["rank"])
    nyt_best.loc[i]["rank_last_week"] = flatten_json(nyt_best.loc[i]["rank_last_week"])
    nyt_best.loc[i]["title"] = flatten_json(nyt_best.loc[i]["title"]) 
    nyt_best.loc[i]["weeks_on_list"] = flatten_json(nyt_best.loc[i]["weeks_on_list"])     
    
nyt_best.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,5b4aa4ead3089013507db18b,2008-05-24,2008-06-08,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",27,Bantam,ODD HOURS,1,0,1
1,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,25.99,"Little, Brown",THE HOST,2,1,3
2,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,24.95,St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2
3,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,22.95,Putnam,THE FRONT,4,0,1
4,5b4aa4ead3089013507db18f,2008-05-24,2008-06-08,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,24.95,Doubleday,SNUFF,5,0,1


In [5]:
#following is mine (CJS)
#do some cleaning and integrity checking
#to deal with typos and variant spellings

#functions for cleaning string-valued columns in dataframes
#for situations where there is a small set of valid strings
# e.g. business names, countries...
#and the strings have to belong to that set
#but there could be typos / alternate spellings
#approach:
# 1. get_matches(...) -> find set of very similar strings using Levenshtein distance
# 2. print and inspect list of matches
# 3. replace_matches(..., checked_matches) -> replace the minority value of each match with majority value

def get_matches(df, colname, levenshtein_cutoff):
    matches = []
    unique_vals = df[colname].unique()
    for i, u in enumerate(unique_vals):
        for j, v in enumerate(unique_vals):
            if(i <= j): continue
            if(lev.distance(u, v)/max(len(u), len(v)) < levenshtein_cutoff):
                matches.append((u, v))
    return matches

def replace_matches(df, colname, matches):
    replacements = []
    temp_list = list(zip(*matches))
    all_match_vals = temp_list[0] + temp_list[1]
    unique_vals = np.unique(all_match_vals)
    counts = np.array([np.sum(df[colname]==v) for v in unique_vals])
    order = np.argsort(counts)[::-1]
    for ind in order:
        u = unique_vals[ind]
        to_remove = []
        for m in matches:
            if(u in m):
                other = [v for v in m if v!=u][0]
                replacements.append((other, u)) #replace <other> with u
                to_remove.append(m)
        #remove at end to avoid errors due to index shift
        for t in to_remove:
            matches.remove(t)
        #actually perform replacements here
        for r in replacements:
            df.loc[df[colname]==r[0], colname] = r[1]


In [6]:
#try to find author names that are very similar - likely typos or variant spellings
# try levenshtein distance of 0.3, turns out this is too permissive
m = get_matches(nyt_best, "author", 0.3)
for t in m: print(t)


('Dean Koontz', 'Dean R Koontz')
('Christopher Moore', 'Christopher Reich')
('Clive Cussler and Jack Du Brul', 'Clive Cussler with Jack Du Brul')
('Carol Higgins Clark', 'Mary Higgins Clark')
('Elizabeth Berg', 'Elizabeth George')
('Elizabeth Berg', 'Elizabeth Peters')
('WEB Griffin and William E Butterworth IV', 'W E B Griffin and William E Butterworth IV')
('Anne Rice', 'Luanne Rice')
('J R Ward', 'JR Ward')
('Dale Brown', 'Dan Brown')
('Stephen White', 'Stephen Hunter')
('Steve Martin', 'Steve Martini')
('JA Jance', 'J A Jance')
('Christopher Bohjalian', 'Chris Bohjalian')
('Robert Harris', 'Robert Crais')
('James Patterson and Mark Sullivan', 'James Patterson and Howard Roughan')
('James Patterson and Mark Sullivan', 'James Patterson and Marshall Karp')
('James Patterson and David Ellis', 'James Patterson and Mark Sullivan')
('Rita Mae Brown and Sneaky Pie Brown', 'Rita Mae Brown & Sneaky Pie Brown')
('Carlos Ruiz Zafon', 'Carlos Ruiz Zafón')
('James Wesley Rawles', 'James Wesley, 

In [7]:
#smaller distance -> fewer false positives
m = get_matches(nyt_best, "author", 0.15)
for t in m: print(t)

('Clive Cussler and Jack Du Brul', 'Clive Cussler with Jack Du Brul')
('WEB Griffin and William E Butterworth IV', 'W E B Griffin and William E Butterworth IV')
('J R Ward', 'JR Ward')
('Steve Martin', 'Steve Martini')
('JA Jance', 'J A Jance')
('Rita Mae Brown and Sneaky Pie Brown', 'Rita Mae Brown & Sneaky Pie Brown')
('Carlos Ruiz Zafon', 'Carlos Ruiz Zafón')
('James Wesley Rawles', 'James Wesley, Rawles')
('John le Carre', 'John le Carré')
('Jo Nesbø', 'Jo Nesbo')
('C  J Box', 'C J Box')
('Catherine Coulter and J T Ellison', 'Catherine Coulter and JT Ellison')
('Jim Harrison', 'Kim Harrison')
('Annie Proulx', 'E Annie Proulx')
('WEB Griffin and William E Butterworth', 'W E B Griffin and William E Butterworth IV')
('WEB Griffin and William E Butterworth', 'WEB Griffin and William E Butterworth IV')
('JD Robb', 'J D Robb')
('CJ Box', 'C J Box')
('BA Paris', 'B A Paris')


In [8]:
#author name matches seem all OK except for 2
m.remove(('Jim Harrison', 'Kim Harrison'))
m.remove(('Steve Martin', 'Steve Martini'))

In [9]:
#now replace the matches
replace_matches(nyt_best, 'author', m)

In [10]:
#check that this worked:
m = get_matches(nyt_best, "author", 0.15)
for t in m: print(t)

('Steve Martin', 'Steve Martini')
('Jim Harrison', 'Kim Harrison')


In [11]:
#these are precisely the two similar names we did not want to replace
#all the other cases are gone
#so everything worked

In [19]:
test_matches = [('C  J Box', 'C J Box')]
replace_matches(nyt_best, 'author', test_matches)

[('C  J Box', 'C J Box')]
[('C  J Box', 'C J Box')]


In [ ]:
nyt_best.loc[nyt_best['author']]

In [ ]:
for t in m: print(t)

In [ ]:
#start check
m = get_matches(nyt_best, "publisher", 0.3)
for t in m: print(t)

In [ ]:
# from kaggle kernel
# add three new columns only with alphanumeric values for comparision and spellcheck detection

raw_books['l_author'] = set_lc_values('author')
raw_books['l_publisher'] = set_lc_values('publisher')
raw_books['l_title'] = set_lc_values('title')

raw_books.head()



In [ ]:
# following is my work
#any given title may appear multiple times
#because it was in multiple weekly bestseller lists
#also, some books may have the same title
# here is an example showing both phenomena:

raw_books.loc[raw_books['title']=="SNUFF",:]


In [ ]:
#we want to find all unique books and assign each an ID
# bookID = <lc author>_<lc title> 

raw_books['book_ID'] = raw_books['l_author'] + '_' + raw_books['l_title']

In [ ]:
print(len(raw_books['book_ID'].unique()))

In [ ]:
ls = [(1,2), (2, 'a')]
x = ls[1]
ls.remove(x)
ls

In [ ]:
#check for IDs that are similar
#because these may be spurious duplications from the same book
#that arise due to typo
possible_matches = []
book_ID_groups = raw_books.groupby('book_ID')
for i, (ID1, fields1) in enumerate(book_ID_groups):
    for j, (ID2, fields2) in enumerate(book_ID_groups):
        if(j <= i): continue
        if(lev.distance(ID1, ID2)/max(len(ID1), len(ID2)) < 0.1):
            print("Possible match:")
            print(ID1)
            print(ID2)
            possible_matches.append((ID1, ID2))
    print("Finished checking "+ID1)
print("Finished checking all IDs")

In [ ]:
#can hand-curate possible matches here...

for (ID1, ID2) in possible_matches:
    (n1, n2) = (np.sum(raw_books['book_ID']==ID) for ID in (ID1, ID2))
    if(n1 > n2): raw_books.loc[raw_books['book_ID']==ID2, 'book_ID'] = ID1
           else: raw_books.loc[raw_books['book_ID']==ID1, 'book_ID'] = ID2

In [ ]:
#now we have hopefully gathered the correct set of entries under the correct book_ID


In [ ]:
np.array([3,2]).shape

In [ ]:
def get_majority_string(ls):
    u = np.unique(ls)
    if(len(u)==1): return u[0]
    else: 
        counts = np.array([np.sum(np.array([s==v for s in ls])) for v in u])
        ind = np.argmax(counts)
        if(len(ind.shape)==0): return u[ind] #if ind is a scalar
        else: return u[ind[0]]

In [ ]:
#do integrity checking on publishers
unique_publishers = raw_books['publisher'].unique()
possible_matches = []
for i, p1 in enumerate(unique_publishers):
    for j, p2 in enumerate(unique_publishers):
        if(i <= j): continue
        if(lev.distance(p1, p2)/max(len(p1), len(p2)) < 0.15):
            print("Possible match:")
            print(p1)
            print(p2)
            possible_matches.append((p1, p2))

In [ ]:
#replace with majority
#go through publishers in order of size (number of records)
#and take all matches
replacements = []
publisher_counts = np.array([np.sum(raw_books["publisher"]==p) for p in unique_publishers])
order = np.argsort(publisher_counts)
for ind in order:
    p1 = unique_publishers[ind]
    to_remove = []
    for t in possible_matches:
        if(p1 in t):
            other = [u for u in t if u!=p1]
            replacements.append((other, p1))
            to_remove.append((other, p1))
    for t in to_remove:
        possible_matches.remove(t)
#actually perform replacements here
for t in replacements:
    raw_books.loc[raw_books['publisher']==t[0], 'publisher'] = t[1]
    

In [ ]:
get_majority_string(['stuff', 'x', 'stuff', 'y', 'x', 'x'])

In [ ]:
print('Total number of titles: %s' % (len(raw_books['title'].unique())))
print('Total number of clean titles: %s' % (len(raw_books['l_title'].unique())))

In [ ]:
raw_books.head()

In [ ]:
for gi in g: print(gi)

In [ ]:
print(raw_books['title'].unique())
print(raw_books['l_title'].unique())

In [ ]:
#find just one clean lower title in data that has two different real titles

spec_title = raw_books.groupby(['l_title', 'title'])

doubleTitle = ''
for name, group in spec_title:
    if doubleTitle == name[0]:
        print('lower title: %s' % (name[0]))
    doubleTitle = name[0]
    


In [ ]:
#find real titles based on single lower case title
filter_titles = raw_books[(raw_books['l_title'] == 'crossfire')].title.unique()
for t in filter_titles:
    print(raw_books[(raw_books['title'] == t)][['author', 'publisher', 'title']].reset_index(drop=True)[:1])

In [ ]:
#so we can conclude that title is clean, there is one case where lc stripped version is the same

In [ ]:
print('Total number of authors: %s' % (len(raw_books['author'].unique())))
print('Total number of clean authors: %s' % (len(raw_books['l_author'].unique())))

In [ ]:
spec_author = raw_books.groupby(['l_author', 'author'])

double_author = ''
for name, group in spec_author:
    if double_author == name[0]:
        print('lower author: %s' % (name[0]))
    double_author = name[0]

In [ ]:
#find real author name based on single lower case author name
filter_auth_col = raw_books[(raw_books['l_author'] == 'baparis')].author.unique()
for t in filter_auth_col:
    print(raw_books[(raw_books['author'] == t)][['author', 'publisher', 'title']].reset_index(drop=True)[:1])

In [ ]:

print('Total number of publisher names: %s' % (len(raw_books['publisher'].unique())))
print('Total number of clean publisher names: %s' % (len(raw_books['l_publisher'].unique())))

In [ ]:
spec_publisher = raw_books.groupby(['l_publisher', 'publisher'])

double_publisher = ''
for name,group in spec_publisher:
    if double_publisher == name[0]:
        print(name[0])
    double_publisher = name[0]


In [ ]:


filter_auth_col = raw_books[(raw_books['l_publisher'] == 'stmartins')].publisher.unique()
for t in filter_auth_col:
    print(raw_books[(raw_books['publisher'] == t)][['publisher']].reset_index(drop=True)[:1])

